# Crypto analyser

### Importing libraries

In [9]:
from binance.client import Client

import os
import pandas as pd

from sqlalchemy import create_engine, inspect

import ipywidgets as widgets

from IPython.display import display
from datetime import datetime, date

import plotly.offline as offline
import plotly.graph_objs as go


offline.init_notebook_mode(connected=True)

### API connection
The historical data regarding crypto-currencies come from www.binance.com.
The API key is required. It is free, and here is how to get it: https://www.binance.com/en/support/faq/360002502072.

In [10]:
binance_api_key = os.environ.get('binance_api_key')
binance_api_secret = os.environ.get('binance_api_secret')
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)

### Database connection & functions

In [11]:
MYSQL_HOSTNAME = 'localhost'
MYSQL_USER = os.environ.get('DB_USER')
MYSQL_PASSWORD = os.environ.get('DB_PASS')
MYSQL_DATABASE = 'cranalyserDB'

connection_string = f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOSTNAME}/{MYSQL_DATABASE}'

engine = create_engine(connection_string)
inspector = inspect(engine)

In [12]:
def table_exist(table: str) -> bool:
    if inspector.has_table(table_name=table): return True
    else: return False

def save_df(df, table: str):
    df.to_sql(table, engine, if_exists='replace')

def get_df(table: str) -> pd.DataFrame:
    df = pd.read_sql_table(table, engine)
    return df

### Downloading data functions

In [24]:
bin_sizes = {'1m': 1, '1h': 60, '1d': 1440}

def update_checker(cr_code, frequency, data):

    """Checks the range of data needed for download.

    Parameters
    -----------
    cr_code : str, crypto-currency code e.g BTCUSDT available in API
    frequency : str, determines the data download interval
    data : pd.DataFrame, currently dataframe existing in database

    Returns
    ----------
    old, new : datatime, points defining the range of data to download"""

    if len(data) > 0: old = data['timestamp'].iloc[-1]
    else: old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    new = pd.to_datetime(binance_client.get_klines(symbol=cr_code, interval=frequency)[-1][0], unit='ms')
    return old, new


def get_from_binance(cr_code, frequency='1h', save_db=False, save_csv=False, return_df=True):
    
    """Downloads data from binance.

    Parameter
    ---------
    cr_code : str, crypto-currency code e.g BTCUSDT available in API
    frequency : str, optional,
        determines the data download interval: 1m, 1h, 1d (default is 1h)
        You can specify available frequency in bin_sizes
    save_db : bool, optional,
        If True new data will be append to database (default is False)
    save_csv : bool, optional,
        If True dataframe will be save to csv (default is False)
    return_df : bool, optional,
        If True dataframe is returned (default is True)

    Returns
    --------
    dataframe (for return_df is True)
    """

    data_set_name = '%s_%s_data' % (cr_code.lower(), frequency)

    if table_exist(data_set_name):
        data_df = get_df(data_set_name)
    else:
        data_df = pd.DataFrame()

    oldest_point, newest_point = update_checker(cr_code, frequency, data_df)
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'):
        print('Downloading all available %s data for %s. Wait...' % (frequency, cr_code))
    else: print('Downloading new available %s data for %s. Wait...!' % (frequency, cr_code))
    data_api = binance_client.get_historical_klines(cr_code, frequency, oldest_point.strftime("%d %b %Y %H:%M:%S"),
                                                    newest_point.strftime("%d %b %Y %H:%M:%S"))
    data_api_df = pd.DataFrame(data_api,
                               columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av',
                                        'trades', 'tb_base_av', 'tb_quote_av', 'ignore'])

    data_api_df['timestamp'] = pd.to_datetime(data_api_df['timestamp'], unit='ms')
    if len(data_df) > 0:
        data_df = data_df[:len(data_df)-1]
        data_df = data_df.append(data_api_df)
        data_df.reset_index(drop=True, inplace=True)

    else: data_df = data_api_df

    data_df.set_index('timestamp', drop=True, inplace=True)

    if save_db:
        save_df(data_df, data_set_name)

    if save_csv:
        data_df.to_csv(data_set_name)

    if return_df:
        return data_df

    print('All data is ready!')

### Create widgets and plots

In [14]:
# Plots creating

def get_time_series_plot(data_fr, cr):
    trace_high = go.Scatter(x=data_fr['timestamp'], y=data_fr['high'],
                            name=f'{cr} high price', line = dict(color = '#6699FF'))
    trace_low = go.Scatter(x=data_fr['timestamp'], y=data_fr['low'],
                           name=f'{cr} low price', line = dict(color = '#FF6633'))
    data = [trace_high, trace_low]
    layout = go.Layout(title=dict(text = f'{cr} price'),
                       xaxis = dict(rangeslider=dict(), type='date'),
                       yaxis=dict(title=' Price in USD'))
    fig = go.Figure(data=data, layout=layout)
    return fig.show()

def get_candlestick_plot(data_fr, cr):
    trace = go.Candlestick(x=data_fr.timestamp,
                           open=data_fr.open,
                           high=data_fr.high,
                           low=data_fr.low,
                           close=data_fr.close,
                           increasing = dict(fillcolor = 'greenyellow',
                                         line = dict(color = 'green',
                                                     width = 3)),

                           decreasing = dict(fillcolor = 'lightcoral'),
                           whiskerwidth=0.4)
    data = [trace]
    layout = go.Layout(title=dict(text = f'{cr} candlestick'),
                       xaxis = dict(rangeslider=dict(), type='date'),
                       yaxis=dict(title=' Price in USD'))
    fig = go.Figure(data=data, layout=layout)
    return fig.show()

In [15]:
# Widgets creating
widget_output = widgets.Output()
plot_output = widgets.Output()
candlestick_output = widgets.Output()

cryptocurrencies = {
    'Bitcoin': 'btcusdt',
    'BinanceCoin': 'bnbusdt',
    'Ethereum': 'ethusdt',
    'Litecoin': 'ltcusdt',
    'XRP': 'xrpusdt'
}


dropdown_currency = widgets.Dropdown(options=list(cryptocurrencies.keys()), description='Currency:')
interval_radio = widgets.RadioButtons(options=['1m', '1h', '1d'], value='1d', description='Data interval:')

dates = pd.date_range(datetime(2017, 8, 17), date.today().strftime("%Y %m %d"), freq='D')
options = [(date.strftime(' %d %b %Y '), date) for date in dates]
index = (0, len(options)-1)

date_range_slider = widgets.SelectionRangeSlider(options=options, index=index,
                                          description='Dates', orientation='horizontal', layout={'width': '600px'})
get_data_button = widgets.Button(description='Get data!')



def get_output(currency, interval, start, stop):
    widget_output.clear_output()
    plot_output.clear_output()
    candlestick_output.clear_output()

    get_from_binance(f'{cryptocurrencies[currency]}'.upper(), interval, save_db=True, return_df=False)
    data_fr = get_df(f'{cryptocurrencies[currency].lower()}_{interval}_data')
    data_fr = data_fr[(data_fr['timestamp'] >= start) & (data_fr['timestamp'] <= stop)]

    with widget_output:
        display(data_fr)

    with plot_output:
         get_time_series_plot(data_fr, currency)

    with candlestick_output:
        get_candlestick_plot(data_fr, currency)


def get_data_button_event_handler(button):
    start = date_range_slider.value[0]
    stop = date_range_slider.value[1]
    get_output(dropdown_currency.value, interval_radio.value, start, stop)

get_data_button.on_click(get_data_button_event_handler)

In [16]:
# Dashboard creating

item_layout = widgets.Layout(margin='0 0 50px 0')

input_widgets = widgets.HBox([interval_radio, dropdown_currency, date_range_slider], layout=item_layout)
input_bt = widgets.HBox([get_data_button], layout=item_layout)

tab = widgets.Tab([widget_output, plot_output, candlestick_output], layout=item_layout)
tab.set_title(0, 'Dataset Exploration')
tab.set_title(1, 'Time series Plot')
tab.set_title(2, 'Candlestick Plot')

dashboard = widgets.VBox([input_widgets, input_bt, tab])

### Dashboard

In [19]:
display(dashboard)


All data is ready!
